# Question 1

### Simulate GARCH(1,1) data

In [ ]:
import numpy as np

def simulate_garch(T, omega_1, omega_2, omega_3):
    np.random.seed(42)  # For reproducibility
    r = np.zeros(T)
    h = np.zeros(T)
    r[0] = np.random.normal(0, np.sqrt(omega_1 / (1 - omega_2 - omega_3)))
    h[0] = omega_1 / (1 - omega_2 - omega_3)
    
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        r[t] = np.random.normal(0, np.sqrt(h[t]))
        
    return r, h

# Parameters (example values)
omega_1 = 0.1
omega_2 = 0.1
omega_3 = 0.8

# Simulate data
T = 1000  # Number of time points
r, h = simulate_garch(T, omega_1, omega_2, omega_3)


### Log-Likelihood & Log-Posterior Functions

The likelihood function for the GARCH model parameters $\omega_1, \omega_2, \omega_3$ given returns $r$ is defined as:
$$
l(\omega_1, \omega_2, \omega_3 \mid r) \propto \prod_{t=1}^T h_t^{-\frac{1}{2}} \exp\left(-\frac{1}{2} \sum_{t=1}^T \frac{r_t^2}{h_t}\right)
$$

The posterior distribution, incorporating independent truncated Normal priors for the parameters, is given by:
$$
\pi(\omega_1, \omega_2, \omega_3 \mid r) \propto \exp\left(-\frac{1}{2} \left(\frac{\omega_1^2}{\sigma^2(\omega_1)} + \frac{\omega_2^2}{\sigma^2(\omega_2)} + \frac{\omega_3^2}{\sigma^2(\omega_3)}\right)\right) \prod_{t=1}^T h_t^{-\frac{1}{2}} \exp\left(-\frac{1}{2} \sum_{t=1}^T \frac{r_t^2}{h_t}\right)
$$

This gives us both the log-likelihood expression and the log-posterior expression:
$$
\log l(\omega_1, \omega_2, \omega_3 \mid r) = -\frac{1}{2} \sum_{t=1}^T \left( \log h_t + \frac{r_t^2}{h_t}\right)
$$

$$
\log \pi(\omega_1, \omega_2, \omega_3 \mid r) = -\frac{1}{2} \left(\frac{\omega_1^2}{\sigma^2(\omega_1)} + \frac{\omega_2^2}{\sigma^2(\omega_2)} + \frac{\omega_3^2}{\sigma^2(\omega_3)}\right) -\frac{1}{2} \sum_{t=1}^T \left( \log h_t + \frac{r_t^2}{h_t}\right)
$$


In [ ]:
def log_likelihood(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    h[0] = omega_1 / (1 - omega_2 - omega_3) if (omega_2 + omega_3) < 1 else 100000  # large number for stability
    
    log_lik = 0
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        log_lik -= 0.5 * (np.log(h[t]) + r[t]**2 / h[t])
    
    return log_lik

def log_posterior(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    log_prior = -0.5 * (omega_1**2 / sigma_1**2 + omega_2**2 / sigma_2**2 + omega_3**2 / sigma_3**2)
    return log_likelihood(r, omega_1, omega_2, omega_3) + log_prior

### Implement the RWMH Sampler

1. Follow the guidelines given in that experiment to implement a random walk Metropo-
lis sampler that targets the posterior distribution of a GARCH model. For the
data, you can use simulated data at first, and then look at the same type of real
data (log-returns computed from exchange rates) in a second time

In [ ]:
def simulate_garch(T, omega_1, omega_2, omega_3):
    np.random.seed(42)  # For reproducibility
    r = np.zeros(T)
    h = np.zeros(T)
    h[0] = omega_1 / (1 - omega_2 - omega_3)
    r[0] = np.random.normal(0, np.sqrt(h[0]))

    
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        r[t] = np.random.normal(0, np.sqrt(h[t]))
        
    return r, h


In [ ]:
# Parameters (example values)
omega_1 = 0.1
omega_2 = 0.1
omega_3 = 0.8

# Simulate data
T = 1000  # Number of time points
r, h = simulate_garch(T, omega_1, omega_2, omega_3)

In [ ]:
def log_likelihood(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    h[0] = omega_1 / (1 - omega_2 - omega_3) if (omega_2 + omega_3) < 1 else 100000  # large number for stability
    
    log_lik = 0
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        log_lik -= 0.5 * (np.log(h[t]) + r[t]**2 / h[t])
    
    return log_lik

def log_posterior(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    log_prior = -0.5 * (omega_1**2 / sigma_1**2 + omega_2**2 / sigma_2**2 + omega_3**2 / sigma_3**2)
    return log_likelihood(r, omega_1, omega_2, omega_3) + log_prior

def proposal(omega_old, sigma_proposal ): # function used in the MH algorithm that will "propose" the new parameter (new omega here)
    # omega_old is the list containing the old omegas, we will use a normal probability distribution to determine the new ones
    omega = np.array([0.0,0.0,0.0])
    omega[0] = np.random.normal(loc= omega_old[0], scale = sigma_proposal)
    omega[1] = np.random.normal(loc= omega_old[1], scale = sigma_proposal)
    omega[2] = np.random.normal(loc= omega_old[2], scale = sigma_proposal)

    return omega

def metropolis_hasting_algorithm(iterations, returns, sigma_proposal, initial_omega, sigma_omega):
    # Let's initialize our variable
    acceptation = np.array([False]*iterations) # we will trough the iterations accept or not the change of value of omega
    omega = initial_omega
    omega_sampling = [omega]
    # Let's apply the algorithm trough all the iterations
    for t in range(iterations):
        # We propose a candidate for omegas
        omega_new = proposal(omega, sigma_proposal)
        # We calculate the acceptance alpha which is ratio of the prior of the new omega over the old omega
        alpha = log_posterior(returns,omega_new[0],omega_new[1],omega_new[2],sigma_omega[0],sigma_omega[1],sigma_omega[2]) - log_posterior(returns,omega[0],omega[1],omega[2],sigma_omega[0],sigma_omega[1],sigma_omega[2])
        # We compare alpha with 1 and an uniform random number
        u = np.random.uniform(0,1)
        if alpha > 0:  # reminder that alpha is a log here so we don't compare with 1 but with 0
            acceptation[t] = True
            omega = omega_new
        if alpha < 0:
            if alpha > np.log(u): # we compare to log u and not u
                acceptation[t]= True
                omega = omega_new # we take the new omega
        omega_sampling.append(omega)
    acceptance_rate = np.sum(acceptation)/len(acceptation)*100
    return acceptance_rate, omega_sampling

**Example on simulated data**

So we are going to test few things.
First, if we get metropolis_omega near of our initial_omega that gave the return.
 Secondly, we will check if the f_tilde function allows to reduce the variance of the estimators while keeping the same expected value
 Finally, we will check the unbiasedness of our estimators that has been proven in one of the article
 

In [ ]:
# Parameters
initial_omega = [0.3, 0.3, 0.3]
true_omega = [0.6, 0.7, 0.3]  # These are the true parameters we want to estimate
sigma_omega = [9, 9, 9]  # Standard deviations for the priors

# Simulate data
returns, _ = simulate_garch(900, *true_omega)

# Metropolis-Hastings settings
iterations = 5000
burn_in = 500
sigma_proposal = 0.01

# Execute the Metropolis-Hastings algorithm
acceptance_rate, omega_samples = metropolis_hasting_algorithm(iterations, returns, sigma_proposal, initial_omega, sigma_omega)

# We extract the parameter samples
omega_samples = np.array(omega_samples)  # Convert list of samples into an array for easier slicing
w1_samples = omega_samples[:, 0]
w2_samples = omega_samples[:, 1]
w3_samples = omega_samples[:, 2]

# We plot the parameter trajectories
plt.figure(figsize=(10, 4))
plt.plot(w1_samples[burn_in:], label='ω1')
plt.plot(w2_samples[burn_in:], label='ω2')
plt.plot(w3_samples[burn_in:], label='ω3')
plt.legend(fontsize=20)
plt.xlabel('Iterations after burn-in', fontsize=20)
plt.ylabel('Parameter values', fontsize=20)
plt.title('Trace Plot of GARCH Parameters', fontsize=20)
plt.grid(True)
plt.show()

As previously discussed, we expect that 
$$\frac{1}{n}\sum_{i=1}^n f(X_i) \to \int f \, d\pi \quad \text{almost surely as } n \to \infty.$$ 
In this instance, we define the identity function as $f$: $f([\omega_1,\omega_2,\omega_3]) = [\omega_1,\omega_2,\omega_3]$. Consequently, we will verify whether the empirical means of these three sequences approach the actual values of the parameters $\omega_1$, $\omega_2$, and $\omega_3$.

In [ ]:
# Computing the average values post burn-in
expected_w1 = np.average(w1_samples[burn_in:])
expected_w2 = np.average(w2_samples[burn_in:])
expected_w3 = np.average(w3_samples[burn_in:])

# Display the calculated mean values
print("Calculated Mean Values:")
print(f"Expected ω1: {expected_w1}")
print(f"Expected ω2: {expected_w2}")
print(f"Expected ω3: {expected_w3}")

Our w1,w2,w3 implemented in the garch_simulation were : [0.1, 0.2, 0.7], therefore we have almost the same figures which means that our algorithm works well.

In [ ]:
# Apply a seaborn style to enhance aesthetics
sns.set_style("whitegrid")

fig, axs = plt.subplots(3, figsize=(12, 18))

# Function to plot a histogram with matplotlib, with enhanced presentation
def custom_hist(data, bins, color, label, ax):
    counts, edges, _ = ax.hist(data, bins=bins, color=color, edgecolor='black', alpha=0.75)
    ax.set_xlabel(label, fontsize=14)
    ax.set_ylabel('Count', fontsize=14)
    return counts, edges

# Plot histograms with detailed customization
_, edges1 = custom_hist(w1_samples[burn_in:], 30, 'steelblue', 'Histogram of ω1', axs[0])
axs[0].set_title('Posterior Distribution of ω1', fontsize=16)

_, edges2 = custom_hist(w2_samples[burn_in:], 50, 'forestgreen', 'Histogram of ω2', axs[1])
axs[1].set_title('Posterior Distribution of ω2', fontsize=16)

_, edges3 = custom_hist(w3_samples[burn_in:], 35, 'firebrick', 'Histogram of ω3', axs[2])
axs[2].set_title('Posterior Distribution of ω3', fontsize=16)

# Fine-tuning the plot layout
plt.tight_layout()
plt.show()

We can see that these histograms of the posterior distribution of w1,w2 w3 are almost histograms from normal distribution. However, they are not perfectly centered with the empirical expected value of omega but they are indeed near of them. We could also see it with boxplot.

In [ ]:
# We are now doing to develop a function that will iterate the metropolis_hasting_algorithm in order to make statistical test and study on the distributions of the omega

def mean_metropolis_algorithm(iterations1):
    expected_w1_list = []
    expected_w2_list = []
    expected_w3_list = []
    for i in range(iterations1):
        results = metropolis_hasting_algorithm(iterations, returns, sigma_proposal, initial_omega, sigma_omega)

        # Check if results structure is as expected
        if not isinstance(results, tuple) or len(results) < 2:
            raise ValueError("The results from the Metropolis-Hasting algorithm do not match the expected format.")

        omega_samples = results[1]  # Assuming the second item is the list of omega samples

        # Prepare lists to hold the current iteration values for w1, w2, w3
        w1_samples = []
        w2_samples = []
        w3_samples = []

        # Extract samples depending on whether they are scalar or array-like
        for sample in omega_samples:
            if np.isscalar(sample):  # Handling cases where the sample might be a single scalar value
                w1_samples.append(sample)  # Assuming scalar results pertain to w1 or need clarification on which omega they relate to
            else:  # Otherwise, handle as arrays
                w1_samples.append(sample[0])
                w2_samples.append(sample[1])
                w3_samples.append(sample[2])

        # Calculate means after burn-in period
        w1_expected = np.mean(w1_samples[burn_in:])
        w2_expected = np.mean(w2_samples[burn_in:])
        w3_expected = np.mean(w3_samples[burn_in:])
        
        expected_w1_list.append(w1_expected)
        expected_w2_list.append(w2_expected)
        expected_w3_list.append(w3_expected)
        
    return [expected_w1_list, expected_w2_list, expected_w3_list]

Example on real date (exchange rate like on the first article on zero variance)

In [ ]:
# Parameters
initial_omega = [0.3, 0.3, 0.3]
true_omega = [0.6, 0.7, 0.3]  # These are the true parameters we want to estimate
sigma_omega = [9, 9, 9]  # Standard deviations for the priors

# Simulate data
returns, _ = simulate_garch(900, *true_omega)

# Metropolis-Hastings settings
iterations = 3000 # less iterations than before
burn_in = 300 # smaller burn_in too
sigma_proposal = 0.01


In [ ]:
mean_met_alg = mean_metropolis_algorithm(20)

In [ ]:
print(mean_met_alg)

In [ ]:
# So now, as we said before we are going to check if the f_tilde function allows to reduce the variance of the estimators while keeping the same expected value
# Thanks to the mean_metropolis_algorithm we can get the variance of our estimators

def variance_estimators(mean_met_alg):
    return statistics.variance(mean_met_alg[0]), statistics.variance(mean_met_alg[1]), statistics.variance(mean_met_alg[2])

# So the variance without the method used in the article is 

print("The variance without the method used in the article is:")
print(f"Variance de ω1,w2,w3 : {variance_estimators(mean_met_alg)}")

# We will see the variance reduction in the next questions

# Now, let's check the unbiasedness of our estimators that has been proven in one of the article.

def bias_estimators(mean_met_alg):
    return statistics.mean(mean_met_alg[0]) - true_omega[0], statistics.mean(mean_met_alg[1]) - true_omega[1], statistics.mean(mean_met_alg[2]) - true_omega[2]

print(f"The bias of the estimators are : {bias_estimators(mean_met_alg)}")

# There is still some bias but it is very small and it may be explicated by the few numbers of iterations of the metropolis_hasting_algorithm, so like it was said in the paper, there is the unbiasedness of the estimators

# To end this part we can finally check the mean squared error of the estimators

mse = {}

# We calculate the MSE for each parameter
for i in range(1,4):
    mse[i] = statistics.mean([(omega - true_omega[i-1])**2 for omega in mean_met_alg[i-1]])

print(mse)

# The mse for both parameters are very small.

**Let's do the example on real data as it is asked in the end of the first question**

In [ ]:
from lxml import etree
import requests
import pandas as pd

def fetch_ecb_data(series_id, start_date, end_date):
    # Define the URL template for accessing ECB data
    url = f"https://sdw-wsrest.ecb.europa.eu/service/data/EXR/D.{series_id}.EUR.SP00.A"

    # Set up headers to specify the desired response format as 'generic' XML
    headers = {
        'Accept': 'text/xml'
    }
    params = {
        'startPeriod': start_date,
        'endPeriod': end_date
    }
    response = requests.get(url, headers=headers, params=params)

    # We parse the XML response
    root = etree.fromstring(response.content)
    ns = {'ns': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/message',
          'common': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/common',
          'generic': 'http://www.sdmx.org/resources/sdmxml/schemas/v2_1/data/generic'}

    # Then we extract data points
    data = []
    for series in root.xpath('.//generic:Series', namespaces=ns):
        for obs in series.xpath('.//generic:Obs', namespaces=ns):
            time = obs.find('.//generic:ObsDimension', namespaces=ns).get('value')
            value = obs.find('.//generic:ObsValue', namespaces=ns).get('value')
            data.append({'Date': time, 'Rate': value})

    # Convert list of dictionaries into a DataFrame
    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'])
    df['rate'] = pd.to_numeric(df['Rate'], errors='coerce')
    return df

In [ ]:
# We will use EUR to GBP exchanges rates on the year 2019 to 2022:
df = fetch_ecb_data('GBP', '2019-01-01', '2022-12-31')
df.head()

In [ ]:
df["rate"] = (df["rate"] - df["rate"].shift(1)) / df["rate"].shift(1)
df = df.dropna()
df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['rate'])
df['rate'] = df['rate'].interpolate() # for the missing values
rates = df["rate"].to_numpy()*100



In [ ]:
rates

In [ ]:
# We will do the same as in the first part of the question with simulated data, we will compare the garch model with the metropolis_hasting samplings

# We fit a garch model with our real datas :
import arch
real_garch = arch.arch_model(rates, vol = 'GARCH', p=1, q=1) # GARCH(1,1)

real_garch = real_garch.fit()

print(real_garch.summary())

In [ ]:
# Now we use our metropolis_hasting_algorithm with our real data, we use the same parameters as for the simulated data : ( the same thing as for simulated data)

# Parameters
initial_omega = [0.3, 0.3, 0.3]
true_omega = [0.3, 0.4, 0.9]  # These are the true parameters we want to estimate
sigma_omega = [9, 9, 9]  # Standard deviations for the priors

# Simulate data
returns = rates

# Metropolis-Hastings settings
iterations = 5000
burn_in = 500
sigma_proposal = 0.01

# Execute the Metropolis-Hastings algorithm
acceptance_rate, omega_samples = metropolis_hasting_algorithm(iterations, returns, sigma_proposal, initial_omega, sigma_omega)

# We extract the parameter samples
omega_samples = np.array(omega_samples)  # Convert list of samples into an array for easier slicing
w1_samples = omega_samples[:, 0]
w2_samples = omega_samples[:, 1]
w3_samples = omega_samples[:, 2]

# We plot the parameter trajectories
plt.figure(figsize=(10, 4))
plt.plot(w1_samples[burn_in:], label='ω1')
plt.plot(w2_samples[burn_in:], label='ω2')
plt.plot(w3_samples[burn_in:], label='ω3')
plt.legend(fontsize=20)
plt.xlabel('Iterations after burn-in', fontsize=20)
plt.ylabel('Parameter values', fontsize=20)
plt.title('Trace Plot of GARCH Parameters', fontsize=20)
plt.grid(True)
plt.show()


In [ ]:
mean_met_alg2 = mean_metropolis_algorithm(20)

In [ ]:
# Let's reuse the function we have implemented for the simulated datas :

# Variance
print(f"Variance de ω1,w2,w3 : {variance_estimators(mean_met_alg2)}")
# Bias
print(f"The bias of the estimators are : {bias_estimators(mean_met_alg2)}")
# MSE
mse = {}
for i in range(1,4):
    mse[i] = statistics.mean([(omega - true_omega[i-1])**2 for omega in mean_met_alg2[i-1]])
print(mse)


We have very small variance, bias and mse as in with simulated datas !

In [ ]:
# We can also check directly the mean of the estimators rather than the bias :

def mean_estimators(mean_met_alg2):
    return statistics.mean(mean_met_alg2[0]), statistics.mean(mean_met_alg2[1]), statistics.mean(mean_met_alg2[2])

print(f"The mean value of the estimators are : {mean_estimators(mean_met_alg2)}") # Recall : true_omega_1 = 0. , true_omega_2 = 0.4, true_omega_3 = 0.9

We can see that we have some very few difference between the expected empirical means and the true values. With real datas of EUR to GBP on 2019-2022, it works less than with our simulated datas.

# Question 2

### Gradient Computations to find Control Variates

The gradient of the log-posterior $\ln \pi$ with respect to each parameter $ \omega_i $ involves contributions from the prior and the likelihood:
$$
\frac{\partial \ln \pi}{\partial \omega_i} = -\frac{\omega_i}{\sigma^2(\omega_i)} - \frac{1}{2} \sum_{t=1}^T \left( \frac{1}{h_t} \frac{\partial h_t}{\partial \omega_i} - \frac{r_t^2}{h_t^2} \frac{\partial h_t}{\partial \omega_i} \right), \quad i = 1, 2, 3
$$

The derivatives of the volatility equation $ h_t $ with respect to the parameters are defined as:
$$
\frac{\partial h_t}{\partial \omega_1} = \frac{1 - \omega_3^{t-1}}{1 - \omega_3} \quad \text{(assuming $ \omega_3 \neq 1 $)}
$$

$$
\frac{\partial h_t}{\partial \omega_2} = r_{t-1}^2 + \omega_3 \frac{\partial h_{t-1}}{\partial \omega_2} \quad \text{for } t > 1 \text{, and 0 else}
$$

$$
\frac{\partial h_t}{\partial \omega_3} = h_{t-1} + \omega_3 \frac{\partial h_{t-1}}{\partial \omega_3} \quad \text{for } t > 1 \text{, and 0 else}
$$


**Error in the article ? Should be $\omega_3^t$:**

*Objective:*
Compute the derivative of $h_t= \omega_1 + \omega_2 r_{t-1}^2 + \omega_3 h_{t-1}$ with respect to $\omega_1$, $\frac{\partial h_t}{\partial \omega_1}$.

*Initial Setup:*
Since $\omega_1$ is a constant addition at each time step $t$,
$ \frac{\partial \omega_1}{\partial \omega_1} = 1 $
and since $r_{t-1}^2$ does not depend on $\omega_1$,
$ \frac{\partial (r_{t-1}^2)}{\partial \omega_1} = 0 $

*Recursive Derivation:*
Starting from the base case at $t=0$, assuming $h_0$ is either non-dependent on $\omega_1$ or set by an initial condition (making it a constant):
$ h_1 = \omega_1 + \omega_2 r_0^2 + \omega_3 h_0 $ gives 
$ \frac{\partial h_1}{\partial \omega_1} = 1 $

At $t=2$:
$ h_2 = \omega_1 + \omega_2 r_1^2 + \omega_3 h_1 $ gives
$ \frac{\partial h_2}{\partial \omega_1} = 1 + \omega_3 \cdot \frac{\partial h_1}{\partial \omega_1} = 1 + \omega_3 $

Proceeding recursively,
$ h_3 = \omega_1 + \omega_2 r_2^2 + \omega_3 h_2 $ gives 
$ \frac{\partial h_3}{\partial \omega_1} = 1 + \omega_3 \cdot \frac{\partial h_2}{\partial \omega_1} = 1 + \omega_3 + \omega_3^2 $

$$\vdots$$

$ \frac{\partial h_t}{\partial \omega_1} = 1 + \omega_3 + \omega_3^2 + \ldots + \omega_3^{t-1} $

Summation as a geometric series for $\omega_3 \neq 1$ yields:
$$ \frac{\partial h_t}{\partial \omega_1} = \frac{1 - \omega_3^t}{1 - \omega_3} $$


In [ ]:
def compute_h_t_derivatives(r, omega_1, omega_2, omega_3):
    T = len(r)
    h = np.zeros(T)
    grad_h_omega1 = np.zeros(T)
    grad_h_omega2 = np.zeros(T)
    grad_h_omega3 = np.zeros(T)

    # Initial volatility
    h[0] = omega_1  

    # Compute h and its derivatives
    for t in range(1, T):
        h[t] = omega_1 + omega_2 * r[t-1]**2 + omega_3 * h[t-1]
        
        # Derivative of h_t with respect to omega_1
        grad_h_omega1[t] = (1 - omega_3**(t)) / (1 - omega_3) if omega_3 != 1 else t  # Using the geometric series

        # Derivative of h_t with respect to omega_2
        grad_h_omega2[t] = r[t-1]**2 + omega_3 * grad_h_omega2[t-1]

        # Derivative of h_t with respect to omega_3
        grad_h_omega3[t] = h[t-1] + omega_3 * grad_h_omega3[t-1]

    return h, grad_h_omega1, grad_h_omega2, grad_h_omega3

def compute_log_posterior_gradients(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    h, grad_h_omega1, grad_h_omega2, grad_h_omega3 = compute_h_t_derivatives(r, omega_1, omega_2, omega_3)
    T = len(r)
    gradients = np.zeros(3)

    # Compute the gradient of the log-posterior for each parameter
    gradients[0] = -omega_1 / sigma_1**2 - 0.5 * np.sum((1 / h) * grad_h_omega1 - (r**2 / h**2) * grad_h_omega1)
    gradients[1] = -omega_2 / sigma_2**2 - 0.5 * np.sum((1 / h) * grad_h_omega2 - (r**2 / h**2) * grad_h_omega2)
    gradients[2] = -omega_3 / sigma_3**2 - 0.5 * np.sum((1 / h) * grad_h_omega3 - (r**2 / h**2) * grad_h_omega3)

    return gradients

We suppose that P is a polynomial : $P(x) = \sum_{j=1}^{3}a_jx_j $.  
Our goal is to find the vector $a = (a_1, a_2, a_3)$ that minimizes the variance of $\widetilde{f} = f + a^Tz$, where $z = -\frac{1}{2}∇ lnπ(x)$.

 


We will now show that it is possible to find the optimal a by doing a linear regression.

$$V(\widetilde{f}) = V(f+a^Tz) = \mathbb{E}[(f+a^Tz)^2] - \mathbb{E}[(f+a^Tz)]^2 = \mathbb{E}[(f+a^Tz)^2] - \mathbb{E}[f]^2 \quad$$ 
The last equality comes from the fact that by construction $\mathbb{E}[\widetilde{f}] = \mathbb{E}[f] $.

We showed that $\mathbb{E}[f]^2$ does not depend by $a$. 
So minimizing $V(\widetilde{f})$ is equivalent to minimizing $\mathbb{E}[(f+a^Tz)^2] = \mathbb{E}[(f- a^T(-z))^2] $.  
We recognize the minimization problem of a linear regression of $f$ on $-z$ so we know that $a = \mathbb{E}[zz^T]^{-1}\mathbb{E}[zf]$ as the article says.

If $ \mathbb{E}[f^2]<∞$ and $\mathbb{E}[[ || -z || ]^2]<∞$ and $\mathbb{E}[zz^T]$ is a non singular matrix then the OLS estimator is consistent (if observations are IID). So we will estimate $a$ by $â$ the OLS estimator.
$$â = (\sum_{t=1}^{T}z_tz_t^T)^{-1}(\sum_{t=1}^{T}z_tf_t)$$

ATTENTION PEUT ETRE QUE NOS  $(z_t,f_t)$ ne sont pas IID.



In [ ]:
def optimal_a(f,z):
    '''
    Calculates the OLS estimator of the vector a

    Args : 
        x (float): The point at which to evaluate the alternative function.
        f (ndarray): Vector containing observed values of the original function f.
        
    Return : OLS estimator of the vector a
    '''
    model = LinearRegression()
    model.fit(f, -z) #performs the linear regression of f on -z
    a = model.coef_
    return a

In [ ]:
def alternative_f(x,f,log_post_gradient):
    '''
    Calculates the value of the alternative function `f_tilde`
    at a given point `x` based on the observed values of the original function `f` and log-posterior 
    gradient `log_post_gradient`.

    Args:
        x (float): The point at which to evaluate the alternative function.
        f (ndarray): Vector containing observed values of the original function f.
        log_post_gradient (ndarray): The log-posterior gradient.

    Return:
        float: The value of the alternative function `f_tilde` at point `x`.
    '''
    z = (-1/2) * log_post_gradient
    a = optimal_a(f,z)
    return f(x)+ np.dot(a.t,z)

Now we will test this method.
As previously discussed, we expect that 
$$\frac{1}{n}\sum_{i=1}^n \widetilde{f}(X_i)  = \frac{1}{n}\sum_{i=1}^n (f(X_i) + a^Tz(X_i)) \to \int \widetilde{f}  \, d\pi \quad \text{almost surely as } n \to \infty.$$ 
$f$ is the same as in question 1: $f([\omega_1,\omega_2,\omega_3]) = [\omega_1,\omega_2,\omega_3]$. Consequently, we will verify whether the empirical means of these three sequences approach the actual values of the parameters $\omega_1$, $\omega_2$, and $\omega_3$ and if the variance is lower.

In [ ]:
# Parameters (example values)
omega_1, omega_2, omega_3 =0.1, 0.1, 0.8
sigma_1, sigma_2, sigma_3 = 9, 9, 9
# Simulate data
T = 1000  # Number of time points
r, h = simulate_garch(T, omega_1, omega_2, omega_3)

In [ ]:
#we have
log_post_gradient = compute_log_posterior_gradients(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3)



In [ ]:
w1_expected = np.mean(alternative_f(w1_samples[burn_in:]))
w2_expected = np.mean(alternative_f(w2_samples[burn_in:]))
w3_expected = np.mean(alternative_f(w3_samples[burn_in:]))

In [ ]:
#compute a 
#on peut oublier cette cellule
#surement pas la meilleure methode, je vais faire un truc mieux avec une regression apres

def optimal_a(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3):
    gradients = compute_log_posterior_gradients(r, omega_1, omega_2, omega_3, sigma_1, sigma_2, sigma_3) 
    a = [0,0,0]
    for t in range(0,gradients[0].size + 1):
        z = (-1/2) * gradients[][t] 
        Sigma_matrix = np.dot(z,z.t)
        sigma_zf = np.dot(z,f)
        a = a + (-np.dot(np.linalg.inv(Sigma_matrix),sigma_zf)) 

return a/gradients[0].size #la moyenne des a 
